In [1]:
from utils import copy_csv_results, extract_summary_stats
from pathlib import Path
import os
import pandas as pd
import plotly.express as px

In [2]:
# Define your output directory (folder containing the results of the analysis in 1_image_analysis)
PARENT_FOLDER = Path("./output")

# Define your username (subfolder name under the output directory)
USERNAME = "Andrew"

# Merge parent_folder and username into a Path
results_directory = PARENT_FOLDER / USERNAME

In [3]:
# Copies .csv files containing per organoid results into a per_organoid_stats folder within results_directory
copy_csv_results(results_directory)

In [4]:
# Define the directory containing the copied per_organoid_stats
csv_results_path = Path(os.path.join(results_directory, "per_organoid_stats"))

# Initialize an empty DataFrame to collect all summary dataframes
final_df = pd.DataFrame()

# Scan for .csv files in each subdirectory
for csv_path in csv_results_path.glob("*.csv"):
    summary_df = extract_summary_stats(csv_path)
    # Append the summary_df to the final_df
    final_df = pd.concat([final_df, summary_df], ignore_index=True)
    
# Create the summary_stats directory if it doesn't exist
summary_stats_directory = Path(results_directory, "summary_stats")
summary_stats_directory.mkdir(parents=True, exist_ok=True)

# Save the final_df as a .csv file under the summary_stats directory
final_csv_path = summary_stats_directory / "summary_stats.csv"
final_df.to_csv(final_csv_path, index=False)

In [5]:
# Drop morphology and class columns
ratios_df = final_df.drop(columns=['Class Name', 'Area', 'Area_filled', 'Perimeter', 'Circularity', 'Eccentricity', 'Solidity'])

# Remove duplicates based on 'well_id' and 'plate_name'
ratios_df = ratios_df.drop_duplicates(subset=['well_id', 'plate_name'])

# Group by plate_name
plate_names = ratios_df['plate_name'].unique()

for plate_name in plate_names:
    # Filter the dataframe for the current plate_name
    df_filtered = ratios_df[ratios_df['plate_name'] == plate_name]
    
    # Plotting
    fig = px.bar(df_filtered, x='well_id', y=['organoid_ratio','spheroid_ratio'], title=f'Organoid/Spheroid ratio per Well for Plate: {plate_name}')
    
    # Construct the file path for saving
    # file_path = summary_stats_directory / f"{plate_name}_organoid_ratio.png"
    
    # Save plot as PNG file
    # fig.write_image(file_path.as_posix(), scale=2)  # Adjust scale as needed for resolution
    
    # Show plot
    fig.show()

c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [6]:
# Group by plate_name
plate_names = final_df['plate_name'].unique()

for plate_name in plate_names:
    # Filter the dataframe for the current plate_name
    df_filtered = final_df[final_df['plate_name'] == plate_name]
    
    # Plotting
    fig = px.scatter(df_filtered, x='well_id', y='Area_filled', color='Class Name', title=f'Object Area per Class for Plate: {plate_name}')
    
    # Construct the file path for saving
    # file_path = summary_stats_directory / f"{plate_name}_object_area.png"
    
    # Save plot as PNG file
    # fig.write_image(file_path.as_posix(), scale=2)  # Adjust scale as needed for resolution
    
    # Show plot
    fig.show()

c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



c:\Users\albertds\mambaforge\envs\int_organoids\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

